# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

In [2]:
# load data from database
engine = create_engine('sqlite:///PipelineDataClean.db')
# Load data from the database into a DataFrame
query = "SELECT * FROM merged_df"  # Replace 'your_table_name' with the actual name of your table
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df)

# Load data from the database into a DataFrame
table_name = 'merged_df'  # Replace 'your_table_name' with the actual name of your table
df = pd.read_sql_table(table_name, engine)

# Define feature variables (X) and target variable (y)
##X = df.drop(columns=['message'])  # Replace 'target_column_name' with the name of your target column
##Y = df['message']  # Replace 'target_column_name' with the name of your target column

# Step 1: Define feature and target variables
X = df['message']  # Message column as feature variable
Y = df.drop(columns=['id', 'message', 'original', 'genre'])  # 36 categories as target variables
#####df = 
##X = message"
##Y = 

          id                                            message  \
0          2  Weather update - a cold front from Cuba that c...   
1          7            Is the Hurricane over or is it not over   
2          8                    Looking for someone but no name   
3          9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4         12  says: west side of Haiti, rest of the country ...   
...      ...                                                ...   
26023  30261  The training demonstrated how to enhance micro...   
26024  30262  A suitable candidate has been selected and OCH...   
26025  30263  Proshika, operating in Cox's Bazar municipalit...   
26026  30264  Some 2,000 women protesting against the conduc...   
26027  30265  A radical shift in thinking came about as a re...   

                                                original   genre  related  \
0      Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                     Cyclone nan fini os

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
##def tokenize(text):
  ##  pass
def tokenize(text):
    """
    Tokenize the input text.

    Args:
    text (str): Input text to be tokenized.

    Returns:
    list: List of tokens.
    """
    # Define punctuation marks
    punctuation = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'''
    
    # Initialize an empty list to store tokens
    tokens = []
    
    # Split the text into words
    words = text.split()
    
    # Iterate through each word
    for word in words:
        # Remove punctuation marks from the word
        word = ''.join([char for char in word if char not in punctuation])
        # Convert the word to lowercase and add it to the list of tokens
        tokens.append(word.lower())
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



# Step 2: Split the dataset
##X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 3: Text preprocessing (you can customize this part)
vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# Step 4: Define pipeline
pipeline = Pipeline([
    ('vect', vectorizer),  # Text vectorization
    ('tfidf', tfidf_transformer),  # TF-IDF transformation
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # MultiOutputClassifier with RandomForestClassifier
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Step 6: Evaluate the model
from sklearn.metrics import classification_report

# Make predictions on test data
Y_pred = pipeline.predict(X_test)

# Report f1 score, precision, and recall for each output category
for i, col in enumerate(Y_test.columns):
    print(f'Report for {col}:')
    print(classification_report(Y_test[col], Y_pred[:, i]))
##Y_pred = pipeline.predict(X_test)
##print(classification_report(Y_test, Y_pred))

Report for related:
              precision    recall  f1-score   support

           0       0.74      0.27      0.40      1247
           1       0.81      0.97      0.88      3959

    accuracy                           0.80      5206
   macro avg       0.77      0.62      0.64      5206
weighted avg       0.79      0.80      0.77      5206

Report for request:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4304
           1       0.87      0.45      0.59       902

    accuracy                           0.89      5206
   macro avg       0.88      0.72      0.77      5206
weighted avg       0.89      0.89      0.88      5206

Report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg    

C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5151
           1       1.00      0.02      0.04        55

    accuracy                           0.99      5206
   macro avg       0.99      0.51      0.52      5206
weighted avg       0.99      0.99      0.98      5206

Report for refugees:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5046
           1       0.57      0.03      0.05       160

    accuracy                           0.97      5206
   macro avg       0.77      0.51      0.52      5206
weighted avg       0.96      0.97      0.96      5206

Report for death:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4981
           1       0.86      0.14      0.24       225

    accuracy                           0.96      5206
   macro avg       0.91      0.57      0.61      5206
weighted avg       0.96      0.96  

C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
#list the available parameters for Pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': None,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__estimator__bootstrap': True,
 'clf__estimator__ccp_alpha': 0.0,
 'clf__estimator__class_weight': None,


In [9]:
from sklearn.model_selection import GridSearchCV

#  Define parameters to tune/improve the model
parameters = {
    'clf_estimator_n_estimators': [50, 100],
}

# Step 6: Initialize GridSearchCV and tune the pipeline with the provided parameters
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)
cv

In [10]:
  params = {
      
       'clf__estimator__n_estimators': [50, 100],

    }

In [11]:
# create grid search object 
cv = GridSearchCV(pipeline, param_grid=params,cv=3,n_jobs=-1,scoring="accuracy")

cv

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1, param_grid={'clf__estimator__n_estimators': [50, 100]},
             scoring='accuracy')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Step 7: Train the model
#cv.fit(X_train, Y_train)
cv.fit(X_train, Y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1, param_grid={'clf__estimator__n_estimators': [50, 100]},
             scoring='accuracy')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

In [14]:
Y_pred = cv.predict(X_test)

# Report f1 score, precision, and recall for each output category
for i, col in enumerate(Y_test.columns):
    print(f'Report for {col}:')
    print(classification_report(Y_test[col], Y_pred[:, i]))

Report for related:
              precision    recall  f1-score   support

           0       0.76      0.28      0.41      1247
           1       0.81      0.97      0.88      3959

    accuracy                           0.81      5206
   macro avg       0.79      0.63      0.65      5206
weighted avg       0.80      0.81      0.77      5206

Report for request:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4304
           1       0.88      0.47      0.61       902

    accuracy                           0.90      5206
   macro avg       0.89      0.73      0.78      5206
weighted avg       0.90      0.90      0.88      5206

Report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg    

C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4942
           1       0.85      0.09      0.16       264

    accuracy                           0.95      5206
   macro avg       0.90      0.54      0.57      5206
weighted avg       0.95      0.95      0.93      5206

Report for search_and_rescue:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5055
           1       1.00      0.03      0.06       151

    accuracy                           0.97      5206
   macro avg       0.99      0.52      0.52      5206
weighted avg       0.97      0.97      0.96      5206

Report for security:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5100
           1       0.00      0.00      0.00       106

    accuracy                           0.98      5206
   macro avg       0.49      0.50      0.49      5206
weighted avg       0.96

C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5150
           1       0.00      0.00      0.00        56

    accuracy                           0.99      5206
   macro avg       0.49      0.50      0.50      5206
weighted avg       0.98      0.99      0.98      5206

Report for other_infrastructure:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4990
           1       0.00      0.00      0.00       216

    accuracy                           0.96      5206
   macro avg       0.48      0.50      0.49      5206
weighted avg       0.92      0.96      0.94      5206

Report for weather_related:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3739
           1       0.87      0.66      0.75      1467

    accuracy                           0.88      5206
   macro avg       0.87      0.81      0.83      5206
weighted avg 

### 9. Export your model as a pickle file

In [15]:
import pickle

# Assuming 'cv' is your trained GridSearchCV object

# Save the model to a file
with open('Tmodel.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [35]:
%run train_classifier.py PipelineDataClean.db Tmodel.pkl

Loading data...
    DATABASE: PipelineDataClean.db
Building model...
Training model...
Evaluating model...
              precision    recall  f1-score   support

           0       0.75      0.30      0.43      1244
           1       0.81      0.97      0.89      3962

    accuracy                           0.81      5206
   macro avg       0.78      0.63      0.66      5206
weighted avg       0.80      0.81      0.78      5206

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4335
           1       0.86      0.45      0.59       871

    accuracy                           0.90      5206
   macro avg       0.88      0.72      0.77      5206
weighted avg       0.89      0.90      0.88      5206

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5184
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5206
   macro avg       0.

C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4743
           1       0.84      0.28      0.42       463

    accuracy                           0.93      5206
   macro avg       0.89      0.64      0.69      5206
weighted avg       0.93      0.93      0.91      5206

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5131
           1       0.89      0.11      0.19        75

    accuracy                           0.99      5206
   macro avg       0.94      0.55      0.59      5206
weighted avg       0.99      0.99      0.98      5206

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5090
           1       0.33      0.01      0.02       116

    accuracy                           0.98      5206
   macro avg       0.66      0.50      0.50      5206
weighted avg       0.96      0.98      0.97      5206

              preci

C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dibeandr\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      5206

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5147
           1       0.00      0.00      0.00        59

    accuracy                           0.99      5206
   macro avg       0.49      0.50      0.50      5206
weighted avg       0.98      0.99      0.98      5206

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5180
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      5206

              preci